In [ ]:
pip install requests beautifulsoup4

In [22]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re
import os
from urllib.parse import urljoin 

# データベースファイル名
DB_NAME = 'github_repos.db'
# スクレイピング対象のURLのベース
BASE_URL = 'https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers'

def create_database():
    """
    SQLiteデータベースを作成し、テーブルを定義します。
    """
    if os.path.exists(DB_NAME):
        print(f"既存のデータベースファイル '{DB_NAME}' を削除します。")
        os.remove(DB_NAME)
        
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()

        cursor.execute('''
            CREATE TABLE repositories (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                repo_name TEXT UNIQUE NOT NULL,
                language TEXT,
                stars INTEGER
            )
        ''')
        
        conn.commit()
        print(f"データベース '{DB_NAME}' とテーブル 'repositories' を作成しました。")
        return conn
    except sqlite3.Error as e:
        print(f"データベース作成エラー: {e}")
        return None

def scrape_github_repos(conn):
    """GitHubからリポジトリ情報をスクレイピングしてDBに保存します。（URL生成によるページネーション対応）"""
    
    if conn is None:
        return

    page_number = 1
    total_repos_scraped = 0
    cursor = conn.cursor()
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # ページネーションをURL生成で処理する
    while True:
        # URLを生成 (page=N パラメータを追加/置換)
        if page_number == 1:
            current_url = BASE_URL
        else:
            # ページ番号をURLに追加するシンプルなロジック
            current_url = f"{BASE_URL}&page={page_number}"
        
        print(f"\n--- 🌐 ページ {page_number} をスクレイピング中 ---")
        print(f"URL: {current_url}")
        
        time.sleep(1) # ページアクセス前の遅延

        try:
            response = requests.get(current_url, headers=headers)
            response.raise_for_status() 
        except requests.exceptions.RequestException as e:
            print(f"HTTPリクエストエラー (ページ {page_number}): {e}")
            break 

        soup = BeautifulSoup(response.content, 'html.parser')
        
        # リポジトリ一覧のコンテナ要素を特定（Box-rowを使用）
        repo_list = soup.find_all('li', class_='Box-row')
        
        if not repo_list:
            if page_number == 1:
                print("⚠️ 警告: 1ページ目でリポジトリ要素が全く見つかりません。セレクタを確認してください。")
            else:
                print(f"このページ ({page_number}) でリポジトリが見つかりませんでした。全件取得完了と見なします。")
            break

        print(f"  - このページで {len(repo_list)} 個のリポジトリを発見しました。")

        # --- リポジトリ情報の抽出とDB保存 ---
        repos_on_page = 0
        for i, repo in enumerate(repo_list):
            
            # 1. リポジトリ名の取得ロジックを強化: href="/google/リポジトリ名" のリンクを直接探す
            repo_link = repo.find('a', href=re.compile(r'/google/[^/]+$')) 
            repo_name = repo_link.text.strip() if repo_link else None

            # リポジトリ名が取得できない場合は、処理をスキップ
            if not repo_name:
                continue

            # 2. 主要な言語
            lang_tag = repo.find('span', itemprop='programmingLanguage')
            language = lang_tag.text.strip() if lang_tag else 'N/A'
            
            # 3. スターの数
            # href="/google/repo-name/stargazers" のリンクを持つ <a> タグ
            star_tag = repo.find('a', href=re.compile(f'/{re.escape(repo_name)}/stargazers'))
            stars_text = star_tag.parent.text.strip() if star_tag and star_tag.parent else '0'

            try:
                stars = int(stars_text.replace(',', ''))
            except ValueError:
                stars = 0
                
            try:
                cursor.execute('''
                    INSERT OR REPLACE INTO repositories (repo_name, language, stars) 
                    VALUES (?, ?, ?)
                ''', (repo_name, language, stars))
                total_repos_scraped += 1
                repos_on_page += 1
            except sqlite3.Error as e:
                print(f"DB挿入エラー for {repo_name}: {e}")

            # 課題の要件: time.sleep(1) を必ず入れること (リポジトリごと)
            time.sleep(1)

        conn.commit()
        
        # 10個すべて取得できた場合のみ次のページに進む
        if repos_on_page == 10:
             page_number += 1
        else:
             # 10個未満しか取得できなかった場合、最終ページと見なす
             break
            
    print(f"\nDBへの保存が完了しました。合計 {total_repos_scraped} 個のリポジトリを取得しました。")


def display_data(conn):
    """保存されたデータをSELECT文で表示します。"""
    
    if conn is None:
        return

    print("\n--- 💾 データベース保存データ (SELECT文による表示) ---")
    
    cursor = conn.cursor()
    
    try:
        cursor.execute("SELECT repo_name, language, stars FROM repositories ORDER BY stars DESC")
        rows = cursor.fetchall()
        
        if not rows:
            print("データがありません。")
            return
            
        # ヘッダー表示
        print(f"{'No.':<4} | {'リポジトリ名':<40} | {'主要な言語':<20} | {'スターの数':>10}")
        print("-" * 80)
        
        # データ表示
        for i, row in enumerate(rows):
            repo_name, language, stars = row
            print(f"{i+1:<4} | {repo_name:<40} | {language:<20} | {stars:>10,}") 

    except sqlite3.Error as e:
        print(f"DB SELECTエラー: {e}")

def main():
    """メイン処理"""
    
    conn = create_database()
    
    if conn:
        print("\n⚠️ 注意: time.sleep(1)がリポジトリごとに適用されるため、処理に数分から十数分かかる可能性があります。")
        
        scrape_github_repos(conn)
        
        display_data(conn)
        
        conn.close()
        print("\n✅ 処理が完了しました。")

if __name__ == "__main__":
    main()

データベース 'github_repos.db' とテーブル 'repositories' を作成しました。

⚠️ 注意: time.sleep(1)がリポジトリごとに適用されるため、処理に数分から十数分かかる可能性があります。

--- 🌐 ページ 1 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 2 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=2
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 3 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=3
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 4 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=4
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 5 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=5
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 6 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=6
  - このページで 10 個のリポジトリ

In [25]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re
import os
import random # ランダムな遅延のためにインポート
from urllib.parse import urljoin 

# データベースファイル名
DB_NAME = 'github_repos.db'
# スクレイピング対象のURLのベース
BASE_URL = 'https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers'

# 【最終修正】遅延時間の最小値と最大値を設定（3秒から10秒のランダムな遅延）
MIN_SLEEP_TIME = 3
MAX_SLEEP_TIME = 10

def create_database():
    """SQLiteデータベースを作成し、テーブルを定義します。"""
    if os.path.exists(DB_NAME):
        print(f"既存のデータベースファイル '{DB_NAME}' を削除します。")
        os.remove(DB_NAME)
        
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()

        cursor.execute('''
            CREATE TABLE repositories (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                repo_name TEXT UNIQUE NOT NULL,
                language TEXT,
                stars INTEGER
            )
        ''')
        
        conn.commit()
        print(f"データベース '{DB_NAME}' とテーブル 'repositories' を作成しました。")
        return conn
    except sqlite3.Error as e:
        print(f"データベース作成エラー: {e}")
        return None

def scrape_github_repos(conn):
    """GitHubからリポジトリ情報をスクレイピングしてDBに保存します。（ランダム遅延対応）"""
    
    if conn is None:
        return

    page_number = 1
    total_repos_scraped = 0
    cursor = conn.cursor()
    
    # 【修正】新しい User-Agent に変更
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15'
    }

    while True:
        # URLを生成
        if page_number == 1:
            current_url = BASE_URL
        else:
            current_url = f"{BASE_URL}&page={page_number}"
        
        print(f"\n--- 🌐 ページ {page_number} をスクレイピング中 ---")
        print(f"URL: {current_url}")
        
        # 【追加】ページアクセス前にランダムな遅延（3〜5秒）
        page_sleep = random.uniform(3, 5)
        print(f"  - ページアクセス前に {page_sleep:.2f}秒待機します。")
        time.sleep(page_sleep) 

        try:
            response = requests.get(current_url, headers=headers)
            response.raise_for_status() 
        except requests.exceptions.RequestException as e:
            # HTTPエラーが発生した場合、ループを終了
            print(f"HTTPリクエストエラー (ページ {page_number}): {e}")
            break 

        soup = BeautifulSoup(response.content, 'html.parser')
        
        repo_list = soup.find_all('li', class_='Box-row')
        
        if not repo_list:
            if page_number > 1:
                print(f"このページ ({page_number}) でリポジトリが見つかりませんでした。全件取得完了と見なします。")
            break

        print(f"  - このページで {len(repo_list)} 個のリポジトリを発見しました。")

        # --- リポジトリ情報の抽出とDB保存 ---
        repos_on_page = 0
        for i, repo in enumerate(repo_list):
            
            repo_link = repo.find('a', href=re.compile(r'/google/[^/]+$')) 
            repo_name = repo_link.text.strip() if repo_link else None

            if not repo_name:
                continue

            lang_tag = repo.find('span', itemprop='programmingLanguage')
            language = lang_tag.text.strip() if lang_tag else 'N/A'
            
            star_tag = repo.find('a', href=re.compile(f'/{re.escape(repo_name)}/stargazers'))
            stars_text = star_tag.parent.text.strip() if star_tag and star_tag.parent else '0'

            try:
                stars = int(stars_text.replace(',', ''))
            except ValueError:
                stars = 0
                
            try:
                cursor.execute('''
                    INSERT OR REPLACE INTO repositories (repo_name, language, stars) 
                    VALUES (?, ?, ?)
                ''', (repo_name, language, stars))
                total_repos_scraped += 1
                repos_on_page += 1
            except sqlite3.Error as e:
                print(f"DB挿入エラー for {repo_name}: {e}")

            # 課題の要件: time.sleep(1) の代わりにランダム遅延を使用
            sleep_duration = random.uniform(MIN_SLEEP_TIME, MAX_SLEEP_TIME)
            time.sleep(sleep_duration)

        conn.commit()
        
        # 10個すべて取得できた場合のみ次のページに進む
        if repos_on_page == 10:
             page_number += 1
        else:
             break
            
    print(f"\nDBへの保存が完了しました。合計 {total_repos_scraped} 個のリポジトリを取得しました。")


def display_data(conn):
    """保存されたデータをSELECT文で表示します。"""
    
    if conn is None:
        return

    print("\n--- 💾 データベース保存データ (SELECT文による表示) ---")
    
    cursor = conn.cursor()
    
    try:
        cursor.execute("SELECT repo_name, language, stars FROM repositories ORDER BY stars DESC")
        rows = cursor.fetchall()
        
        if not rows:
            print("データがありません。")
            return
            
        # ヘッダー表示
        print(f"{'No.':<4} | {'リポジトリ名':<40} | {'主要な言語':<20} | {'スターの数':>10}")
        print("-" * 80)
        
        # データ表示
        for i, row in enumerate(rows):
            repo_name, language, stars = row
            print(f"{i+1:<4} | {repo_name:<40} | {language:<20} | {stars:>10,}") 

    except sqlite3.Error as e:
        print(f"DB SELECTエラー: {e}")

def main():
    """メイン処理"""
    
    conn = create_database()
    
    if conn:
        print(f"\n⚠️ 注意: time.sleep({MIN_SLEEP_TIME}〜{MAX_SLEEP_TIME})秒のランダム遅延がリポジトリごとに適用されます。完了まで時間がかかります。")
        
        scrape_github_repos(conn)
        
        display_data(conn)
        
        conn.close()
        print("\n✅ 処理が完了しました。")

if __name__ == "__main__":
    main()

データベース 'github_repos.db' とテーブル 'repositories' を作成しました。

⚠️ 注意: time.sleep(3〜10)秒のランダム遅延がリポジトリごとに適用されます。完了まで時間がかかります。

--- 🌐 ページ 1 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers
  - ページアクセス前に 4.92秒待機します。
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 2 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=2
  - ページアクセス前に 3.11秒待機します。
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 3 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=3
  - ページアクセス前に 4.96秒待機します。
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 4 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=4
  - ページアクセス前に 3.88秒待機します。
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 5 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=5
  - ページアクセス前に 3.21秒待機します。
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ

In [26]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re
import os
import random 
from urllib.parse import urljoin 

# データベースファイル名
DB_NAME = 'github_repos.db'
# スクレイピング対象のURLのベース
BASE_URL = 'https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers'

# 遅延時間の最小値と最大値を設定（3秒から10秒のランダムな遅延）
MIN_SLEEP_TIME = 3
MAX_SLEEP_TIME = 10

def create_database():
    """SQLiteデータベースを作成し、テーブルを定義します。（既存DBの削除を停止）"""
    # 【修正】既存のDBファイルを削除するロジックをコメントアウトし、保持する
    # if os.path.exists(DB_NAME):
    #     print(f"既存のデータベースファイル '{DB_NAME}' を削除します。")
    #     os.remove(DB_NAME)
        
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()

        # テーブルが存在しない場合のみ作成
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS repositories (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                repo_name TEXT UNIQUE NOT NULL,
                language TEXT,
                stars INTEGER
            )
        ''')
        
        conn.commit()
        
        # 初回実行時以外は削除メッセージを出さない
        if os.path.exists(DB_NAME):
             print(f"既存のデータベース '{DB_NAME}' を使用し、続きから取得を試みます。")
        else:
             print(f"データベース '{DB_NAME}' とテーブル 'repositories' を作成しました。")
        
        return conn
    except sqlite3.Error as e:
        print(f"データベース作成エラー: {e}")
        return None

def scrape_github_repos(conn):
    """GitHubからリポジトリ情報をスクレイピングしてDBに保存します。（ランダム遅延対応）"""
    
    if conn is None:
        return

    page_number = 1
    total_repos_scraped = 0
    cursor = conn.cursor()
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version=14.0.3 Safari/605.1.15'
    }
    
    # DBに既に保存されているリポジトリ数を取得
    cursor.execute("SELECT COUNT(repo_name) FROM repositories")
    initial_count = cursor.fetchone()[0]
    print(f"📥 現在DBには {initial_count} 個のリポジトリが保存されています。")

    while True:
        # URLを生成
        if page_number == 1:
            current_url = BASE_URL
        else:
            current_url = f"{BASE_URL}&page={page_number}"
        
        print(f"\n--- 🌐 ページ {page_number} をスクレイピング中 ---")
        print(f"URL: {current_url}")
        
        page_sleep = random.uniform(3, 5)
        print(f"  - ページアクセス前に {page_sleep:.2f}秒待機します。")
        time.sleep(page_sleep) 

        try:
            response = requests.get(current_url, headers=headers)
            response.raise_for_status() 
        except requests.exceptions.RequestException as e:
            print(f"HTTPリクエストエラー (ページ {page_number}): {e}")
            break 

        soup = BeautifulSoup(response.content, 'html.parser')
        
        repo_list = soup.find_all('li', class_='Box-row')
        
        if not repo_list:
            if page_number > 1:
                print(f"このページ ({page_number}) でリポジトリが見つかりませんでした。全件取得完了と見なします。")
            break

        print(f"  - このページで {len(repo_list)} 個のリポジトリを発見しました。")

        # --- リポジトリ情報の抽出とDB保存 ---
        repos_on_page = 0
        for i, repo in enumerate(repo_list):
            
            repo_link = repo.find('a', href=re.compile(r'/google/[^/]+$')) 
            repo_name = repo_link.text.strip() if repo_link else None

            if not repo_name:
                # 【修正】リポジトリ名が取得できなかった場合、サイレントスキップせずログを出す
                print(f"  ⚠️ 警告: {i+1}番目の要素でリポジトリ名の取得に失敗し、スキップしました。")
                continue

            lang_tag = repo.find('span', itemprop='programmingLanguage')
            language = lang_tag.text.strip() if lang_tag else 'N/A'
            
            star_tag = repo.find('a', href=re.compile(f'/{re.escape(repo_name)}/stargazers'))
            stars_text = star_tag.parent.text.strip() if star_tag and star_tag.parent else '0'

            try:
                stars = int(stars_text.replace(',', ''))
            except ValueError:
                stars = 0
                
            try:
                # INSERT OR REPLACE INTO を使用することで、既にDBにあるリポジトリはスキップ（更新）される
                cursor.execute('''
                    INSERT OR REPLACE INTO repositories (repo_name, language, stars) 
                    VALUES (?, ?, ?)
                ''', (repo_name, language, stars))
                total_repos_scraped += 1
                repos_on_page += 1
            except sqlite3.Error as e:
                print(f"DB挿入エラー for {repo_name}: {e}")

            sleep_duration = random.uniform(MIN_SLEEP_TIME, MAX_SLEEP_TIME)
            time.sleep(sleep_duration)

        conn.commit()
        
        if repos_on_page == 10:
             page_number += 1
        else:
             break
            
    print(f"\nDBへの保存が完了しました。合計 {total_repos_scraped} 個の新しいリポジトリを取得しました。")


def display_data(conn):
    """保存されたデータをSELECT文で表示します。"""
    
    if conn is None:
        return

    print("\n--- 💾 データベース保存データ (SELECT文による表示) ---")
    
    cursor = conn.cursor()
    
    try:
        cursor.execute("SELECT repo_name, language, stars FROM repositories ORDER BY stars DESC")
        rows = cursor.fetchall()
        
        if not rows:
            print("データがありません。")
            return
            
        print(f"**データベース総件数: {len(rows)} 件**")
        print(f"{'No.':<4} | {'リポジトリ名':<40} | {'主要な言語':<20} | {'スターの数':>10}")
        print("-" * 80)
        
        for i, row in enumerate(rows):
            repo_name, language, stars = row
            print(f"{i+1:<4} | {repo_name:<40} | {language:<20} | {stars:>10,}") 

    except sqlite3.Error as e:
        print(f"DB SELECTエラー: {e}")

def main():
    """メイン処理"""
    
    conn = create_database()
    
    if conn:
        print(f"\n⚠️ 注意: time.sleep({MIN_SLEEP_TIME}〜{MAX_SLEEP_TIME})秒のランダム遅延がリポジトリごとに適用されます。完了まで時間がかかります。")
        
        scrape_github_repos(conn)
        
        display_data(conn)
        
        conn.close()
        print("\n✅ 処理が完了しました。")

if __name__ == "__main__":
    main()

既存のデータベース 'github_repos.db' を使用し、続きから取得を試みます。

⚠️ 注意: time.sleep(3〜10)秒のランダム遅延がリポジトリごとに適用されます。完了まで時間がかかります。
📥 現在DBには 13 個のリポジトリが保存されています。

--- 🌐 ページ 1 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers
  - ページアクセス前に 3.91秒待機します。
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 2 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=2
  - ページアクセス前に 4.74秒待機します。
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 3 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=3
  - ページアクセス前に 4.85秒待機します。
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 4 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=4
  - ページアクセス前に 4.47秒待機します。
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 5 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=5
  - ページアクセス前に 3.65秒待機します。
  - このページで 10 個のリポジ